In [5]:
import nltk
import string
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from pprint import pprint
from unidecode import unidecode
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
import warnings
from tqdm._tqdm_notebook import tqdm_notebook
from scipy.spatial import distance
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
import tensorflow as tf
import transformers
from sent2vec.vectorizer import Vectorizer as S2vectorizer
tqdm_notebook.pandas()
warnings.filterwarnings("ignore")

pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', 0)

## NER Generation

In [6]:
pprint([(X.text, X.label_) for X in nlp("Beyonce is a woman").ents])

[('Beyonce', 'GPE')]


In [7]:
def generate_ners(text):
    result = [unidecode(X.text.lower()) for X in nlp(text).ents]
    return result

In [8]:
df = pd.read_csv('../data/SQuAD_csv.csv', encoding='utf-8').loc[:, ['context']]
df = df.drop_duplicates(subset=['context'])
df['context_ner'] = df['context'].progress_apply(generate_ners)
# df['context'] = df['context'].apply(lambda x: str(x).lower())

In [9]:
df.head()

,context,context_ner
0,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".","[beyonce giselle knowles-carter, september 4, 1981, american, houston, texas, the late 1990s, r&b, destiny, mathew knowles, beyonce, 2003, five, 100, one, crazy in love, baby boy]"
15,"Following the disbandment of Destiny's Child in June 2005, she released her second solo album, B'Day (2006), which contained hits ""Déjà Vu"", ""Irreplaceable"", and ""Beautiful Liar"". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for ""Single Ladies (Put a Ring on It)"". Beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her critically acclaimed fifth studio album, Beyoncé (2013), was distinguished from previous releases by its experimental production and exploration of darker themes.","[june 2005, second, b'day, 2006, ""deja vu"", irreplaceable, beautiful liar, beyonce, golden globe, dreamgirls, 2006, the pink panther, obsessed, 2009, jay z, etta james, cadillac records, 2008, third, sasha fierce, 2008, sasha fierce, six, grammy awards, 2010, song, the year, single ladies, beyonce, 2010, fourth, 2011, 1970s, 1980s, 1990s, fifth, beyonce, 2013]"
27,"A self-described ""modern-day feminist"", Beyoncé creates songs that are often characterized by themes of love, relationships, and monogamy, as well as female sexuality and empowerment. On stage, her dynamic, highly choreographed performances have led to critics hailing her as one of the best entertainers in contemporary popular music. Throughout a career spanning 19 years, she has sold over 118 million records as a solo artist, and a further 60 million with Destiny's Child, making her one of the best-selling music artists of all time. She has won 20 Grammy Awards and is the most nominated woman in the award's history. The Recording Industry Association of America recognized her as the Top Certified Artist in America during the 2000s decade. In 2009, Billboard named her the Top Radio Songs Artist of the Decade, the Top Female Artist of the 2000s and their Artist of the Millennium in 2011. Time listed her among the 100 most influential people in the world in 2013 and 2014. Forbes magazine also listed her as the most powerful female musician of 2015.","[modern-day, beyonce, 19 years, over 118 million, 60 million, destiny, 20, grammy awards, the recording industry association of america, the top certified artist, america, the 2000s decade, 2009, billboard, the top female artist, the 2000s, their artist of the millennium, 2011, time, 100, 2013, 2014, forbes, 2015]"
39,"Beyoncé Giselle Knowles was born in Houston, Texas, to Celestine Ann ""Tina"" Knowles (née Beyincé), a hairdresser and salon owner, and Mathew Knowles, a Xerox sales manager. Beyoncé's name is a tribute to her mother's maiden name. Beyoncé's younger sister Solange is also a singer and a former member of 

## Text Preprocessing

In [10]:
stop_words = stopwords.words("english")

def clean_normalcase_stop(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\s{2,}', " ", text)
    text = unidecode(text)
    return text

def clean_normalcase_nostop(text):
    text = ' '.join([word for word in text.split() if word not in stop_words])
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\s{2,}', " ", text)
    text = unidecode(text)
    return text

def clean_lowercase_stop(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\s{2,}', " ", text)
    text = unidecode(text)
    return text.lower()

def clean_lowercase_nostop(text):
    text = ' '.join([word for word in text.split() if word not in stop_words])
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\s{2,}', " ", text)
    text = unidecode(text)
    return text.lower()

In [11]:
df['cleaned_normalcase_stop'] = df['context'].progress_apply(clean_normalcase_stop)

In [12]:
df['cleaned_normalcase_nostop'] = df['context'].progress_apply(clean_normalcase_nostop)

In [13]:
df['cleaned_lowercase_stop'] = df['context'].progress_apply(clean_lowercase_stop)

In [14]:
df['clean_lowercase_nostop'] = df['context'].progress_apply(clean_lowercase_nostop)

In [15]:
df.head(10)

,context,context_ner,cleaned_normalcase_stop,cleaned_normalcase_nostop,cleaned_lowercase_stop,clean_lowercase_nostop
0,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".","[beyonce giselle knowles-carter, september 4, 1981, american, houston, texas, the late 1990s, r&b, destiny, mathew knowles, beyonce, 2003, five, 100, one, crazy in love, baby boy]",Beyonce Giselle Knowles Carter bi:'janseI bee YON say born September 4 1981 is an American singer songwriter record producer and actress Born and raised in Houston Texas she performed in various singing and dancing competitions as a child and rose to fame in the late 1990s as lead singer of R B girl group Destiny s Child Managed by her father Mathew Knowles the group became one of the world s best selling girl groups of all time Their hiatus saw the release of Beyonce s debut album Dangerously in Love 2003 which established her as a solo artist worldwide earned five Grammy Awards and featured the Billboard Hot 100 number one singles Crazy in Love and Baby Boy,Beyonce Giselle Knowles Carter bi:'janseI bee YON say born September 4 1981 American singer songwriter record producer actress Born raised Houston Texas performed various singing dancing competitions child rose fame late 1990s lead singer R B girl group Destiny s Child Managed father Mathew Knowles group became one world s best selling girl groups time Their hiatus saw release Beyonce s debut album Dangerously Love 2003 established solo artist worldwide earned five Grammy Awards featured Billboard Hot 100 number one singles Crazy Love Baby Boy,beyonce giselle knowles carter bi:'jansei bee yon say born september 4 1981 is an american singer songwriter record producer and actress born and raised in houston texas she performed in various singing and dancing competitions as a child and rose to fame in the late 1990s as lead singer of r b girl group destiny s child managed by her father mathew knowles the group became one of the world s best selling girl groups of all time their hiatus saw the release of beyonce s debut album dangerously in love 2003 which established her as a solo artist worldwide earned five grammy awards and featured the billboard hot 100 number one singles crazy in love and baby boy,beyonce giselle knowles carter bi:'jansei bee yon say born september 4 1981 american singer songwriter record producer actress born raised houston texas performed various singing dancing competitions child rose fame late 1990s lead singer r b girl group destiny s child managed father mathew knowles group became one world s best selling girl groups time their hiatus saw release beyonce s debut album dangerously love 2003 established solo artist worldwide earned five grammy awards featured billboard hot 100 number one singles crazy love baby boy
15,"Following the disbandment of Destiny's Child in June 2005, she released her second solo album, B'Day (2006), which contained hits ""Déjà Vu"", ""Irreplaceable"", and ""Beautiful Liar"". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a re

In [17]:
df.to_pickle('cleaned_ner_23_mar.pkl')

## Read .pkl File

In [ ]:
#Read the pickle file
df2 = pd.read_pickle('cleaned_ner_23_mar.pkl')

## TF-IF Context Querying

In [ ]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(df['cleaned_context'])

In [ ]:
print(tfidf.shape)

In [ ]:
def get_similar_doc(tfidfvectorizer, docs_tfidf, query):
    """
    vectorizer: TfIdfVectorizer model
    docs_tfidf: tfidf vectors for all docs
    query: query doc

    return: doc with highest tf-idf cosine similarity
    """
    
    query_tfidf = tfidfvectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    cosineSimilarities = np.array(cosineSimilarities)
    return cosineSimilarities.argsort()[-3:][::-1]

def get_relevant_sentence(doc_ids, query):
    docs = df.iloc[doc_ids, df.columns.get_loc('context')].apply(lambda x: [i.strip() for i in x.split('.') if len(i)>1])
    sentences = []
    for doc in docs:
        for sentence in doc:
            sentences.append(sentence)

    vectorizer = S2vectorizer()
    vectorizer.bert([query] + sentences)
    vectors = vectorizer.vectors
    query_vec, other_vec = vectors[0], vectors[1:]
    
    print("----- Sentences Retrieved: -----")
    for idx, sentence in enumerate(sentences):
        print(f"{idx}. {sentence}")
    
    distances = distance.cdist([query_vec], other_vec, "cosine")[0]
    min_index = np.argmin(distances)
#     min_distance = distances[min_index]
#     max_similarity = 1 - min_distance
    return sentences[min_index]
    
def get_answer(query):
    query = re.sub('[%s]' % re.escape(string.punctuation), '', query)
    query = re.sub('\s{2,}', " ", query).lower()
    most_similar_doc_ids = get_similar_doc(vectorizer, tfidf, query)
    return get_relevant_sentence(most_similar_doc_ids, query)

In [ ]:
get_answer("when did the group release their multi-platinum second album The Writing's on the Wall?")

## Gigi

In [ ]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

def get_similar_doc(tfidfvectorizer, docs_tfidf, query):
    """
    vectorizer: TfIdfVectorizer model
    docs_tfidf: tfidf vectors for all docs
    query: query doc

    return: doc with highest tf-idf cosine similarity
    """
    
    query_tfidf = tfidfvectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    cosineSimilarities = np.array(cosineSimilarities)
    return cosineSimilarities.argsort()[-3:][::-1]

def get_sentences(doc_ids):
    docs = df.iloc[doc_ids, df.columns.get_loc('context')].apply(lambda x: [i.strip() for i in x.split('.') if len(i)>1])
    print(docs)
    sentences = []
    for doc in docs:
        for sentence in doc:
            sentences.append(sentence)
    return sentences

def infersent(sentences, query_vec):
    similarity = []
    for sent in sentences:
        sim = cosine(query_vec, model.encode([sent])[0])
        similarity.append((sent, sim))

    return similarity
# .sort(key=lambda x: x[1], reverse=True)

    
def get_answer(query):
    query = re.sub('[%s]' % re.escape(string.punctuation), '', query)
    query = re.sub('\s{2,}', " ", query).lower()
    most_similar_doc_ids = get_similar_doc(vectorizer, tfidf, query)
    sentences = get_sentences(most_similar_doc_ids)
    model.build_vocab(sentences, tokenize=True)
    query_vec = model.encode(query)[0]
    
    answers = infersent(sentences, query_vec)
#     print(sorted(answers, key=lambda x: x[1]))
    return sorted(answers, key=lambda x: x[1], reverse=True)


In [ ]:
! mkdir encoder
! curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl
  
! mkdir GloVe
! curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
! unzip GloVe/glove.840B.300d.zip -d GloVe/

In [ ]:
from models import InferSent
import torch

V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = 'GloVe/glove.840B.300d.txt'
model.set_w2v_path(W2V_PATH)

In [ ]:
get_answer("when did the group release their multi-platinum second album The Writing's on the Wall?")

In [ ]:
get_answer("What areas did Beyonce compete in when she was growing up?")